# Notebook do Projeto do Aluno 
## Matéria: Projeto Integrador II

## O Problema de Negócio

- Contexto:

A indústria de aviação comercial tem enfrentado desafios crescentes em um mercado altamente competitivo e sensível a preços. A decisão de compra de uma passagem aérea é influenciada por diversos fatores, como data da viagem, demanda, políticas de preços das companhias aéreas e até condições econômicas. Além disso, as passagens aéreas são um dos produtos mais dinâmicos em termos de variação de preço, com tarifas mudando frequentemente, conforme a demanda e a proximidade da data de voo.

Atualmente, muitas empresas que fornecem serviços de vendas de passagens aéreas agregam as ofertas de várias companhias, permitindo que os clientes comparem e comprem passagens pelo menor preço. No entanto, essa variação de preços dificulta tanto a precificação quanto a otimização de receita. A falta de previsibilidade no preço das passagens faz com que a empresa tenha dificuldades em oferecer os melhores preços no momento adequado, o que pode resultar em perdas de oportunidades de vendas ou margens reduzidas.

- Objetivo do Desafio:

Neste cenário, você trabalha como Cientista de Dados para uma empresa fictícia de venda de passagens aéreas online. O CEO da empresa está preocupado com a imprevisibilidade dos preços das passagens, o que tem impactado a capacidade de oferecer as melhores tarifas e, ao mesmo tempo, manter a lucratividade. A empresa atualmente enfrenta perdas devido a margens reduzidas causadas pela falta de um processo preditivo eficiente para ajuste de preços.

O objetivo deste projeto é desenvolver um modelo de Machine Learning que seja capaz de prever com precisão o preço das passagens aéreas, considerando várias features relacionadas à data da viagem, rota, demanda, entre outros fatores. Com uma previsão precisa dos preços, a empresa poderá ajustar suas ofertas de maneira proativa, maximizando a conversão de vendas e a lucratividade.

Suponha que o erro médio no preço das passagens, sem um modelo preditivo, resulta em uma perda financeira estimada de $5000 por venda, devido à oferta de preços inadequados (acima ou abaixo do valor ótimo). Com uma estimativa de 200.000 vendas no próximo ano, essa ineficiência representa uma perda potencial de $1Bi. A implementação de um modelo com um RMSE mais baixo poderá reduzir significativamente essa perda, aumentando as margens de lucro da empresa.

- Baseline:

Perda financeira por venda: $ 5000

RMSE atual =  desvio-padrão da base


- Variáveis:

Companhia Aérea: O nome da companhia aérea está armazenado na coluna de companhia aérea. É uma feature categórica com 6 companhias aéreas diferentes.

Voo: Armazena informações sobre o código do voo do avião. É uma feature categórica.

Cidade de Origem: A cidade de onde o voo decola. É uma feature categórica com 6 cidades únicas.

Hora de Partida: Esta é uma feature categórica derivada, criada ao agrupar os períodos de tempo em intervalos. Armazena informações sobre o horário de partida e tem 6 rótulos de tempo únicos.

Escalas: Uma feature categórica com 3 valores distintos que armazena o número de escalas entre as cidades de origem e destino.

Hora de Chegada: Esta é uma feature categórica derivada, criada ao agrupar intervalos de tempo em faixas. Possui seis rótulos de tempo distintos e mantém informações sobre o horário de chegada.

Cidade de Destino: A cidade onde o voo vai pousar. É uma feature categórica com 6 cidades únicas.

Classe: Uma feature categórica que contém informações sobre a classe dos assentos; possui dois valores distintos: Executiva e Econômica.

Duração: Uma feature contínua que mostra o tempo total de viagem entre as cidades, em horas.

Dias Restantes: Esta é uma característica derivada, calculada subtraindo a data da viagem pela data da reserva.

Preço: A variável alvo que armazena informações sobre o preço do bilhete.

## Imports:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from boruta import BorutaPy
from IPython.display import Image, HTML
from datetime import datetime
from geopy.distance import geodesic
from sklearn.preprocessing import RobustScaler, LabelEncoder, MinMaxScaler
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from lightgbm import LGBMRegressor
from bayes_opt import BayesianOptimization

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)


## Funções:

## Dataset:

In [30]:
df_raw = pd.read_csv('data/dados_passagens.csv')

In [31]:
df_raw.rename(columns={'Unnamed: 0': 'desconhecido', 
                       'date': 'data_voo', 
                       'airline': 'cia_aerea', 
                       'ch_code': 'ch_code', 
                       'num_code': 'num_code', 
                       'dep_time': 'hora_partida',
                       'from': 'cidade_origem', 
                       'time_taken': 'duracao', 
                       'stop': 'escalas', 
                       'arr_time': 'hora_chegada', 
                       'to': 'cidade_destino', 
                       'price': 'preco', 
                       'days_left': 'dias_restantes'}, inplace=True)

In [32]:
#criando a coluna de código do vôo:
df_raw['cd_voo'] = df_raw['ch_code'] + df_raw['num_code'].astype(str)

#excluindo as colunas de ch_code e num_code:
df_raw.drop(columns = ['ch_code', 'num_code'], inplace = True)

## 1.0 Descrição dos Dados:

In [57]:
df1 = df_raw.copy()

### 1.1 Dimensão dos Dados

In [34]:
print("Quantidade de linhas: {}".format(df1.shape[0]))
print("Quantidade de colunas: {}".format(df1.shape[1]))

Quantidade de linhas: 206774
Quantidade de colunas: 12


### 1.2 Tipos de Dados

In [35]:
df1.dtypes

desconhecido       int64
data_voo          object
cia_aerea         object
hora_partida      object
cidade_origem     object
duracao           object
escalas           object
hora_chegada      object
cidade_destino    object
preco             object
dias_restantes     int64
cd_voo            object
dtype: object

### 1.3 Checando Dados Nulos

In [36]:
df1.isna().sum()

desconhecido      0
data_voo          0
cia_aerea         0
hora_partida      0
cidade_origem     0
duracao           0
escalas           0
hora_chegada      0
cidade_destino    0
preco             0
dias_restantes    0
cd_voo            0
dtype: int64

### 1.4 Ajustando os Tipos de Dados

In [ ]:
df1['preco'] = df1['preco'].astype(str).str.replace(',', '', regex=False).astype(int)
df1['dias_restantes'] = df1['dias_restantes'].astype('int64')
df1['data_voo'] = pd.to_datetime(df1['data_voo'], format='%d-%m-%Y')
df1['hora_partida'] = pd.to_datetime(df1['hora_partida'], format='%H:%M').dt.time
df1['hora_chegada'] = pd.to_datetime(df1['hora_chegada'], format='%H:%M').dt.time
df1['duracao'] = pd.to_timedelta(
    df1['duracao'].str.replace('h', ' hours ').str.replace('m', ' minutes'),
    errors='coerce')